# 종합

#### 필요한 라이브러리 import

In [1]:
import requests # 웹 페이지 소스를 얻기 위한 패키지(기본 내장 패키지이다.)
from bs4 import BeautifulSoup # 웹 페이지 소스를 얻기 위한 패키지, 더 간단히 얻을 수 있다는 장점이 있다고 한다.(!pip install beautifulsoup4 으로 다운받을 수 있다.)
import pandas as pd # 데이터를 처리하기 위한 가장 기본적인 패키지
import time # 사이트를 불러올 때, 작업 지연시간을 지정해주기 위한 패키지이다. (사이트가 늦게 켜지면 에러가 발생하기 때문)

In [19]:
base_url = 'http://dontb.co.kr'
start_url = 'http://dontb.co.kr/product/list.html?cate_no=12'

page_url = start_url + '&page=' + '{}' 
# 던티비 또한 상품의 페이지 수가 증가할 때마다 &page=1 , &page=2 , &page=3 형식으로 일정한 패턴을 가진다.

page_1 = requests.get(start_url, headers = {'user-agent' : 
                                            ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                       'Chrome/71.0.3578.98 Safari/537.36')})
# 클라이언트는 서버에 요청을 보낼 때 여러 가지 부가 정보를 보낸다. 
# 예를 들면 A라는 사이트에서 링크를 눌러 B라는 사이트로 접속할 경우에, B라는 사이트에 A를 리퍼러(Referer)로 전달하는 것이다.
# 일부 웹 사이트의 경우에는 리퍼러를 이용해 접속을 허용하거나 차단하기도 한다.
# 던티비 사이트는 웹페이지에 자동화된 소프트웨어로 의한 접근을 제한하고 있다. 
# 이 때문에 그냥 접속하면 에러가 발생하는데, 이 때 유저에이전트를 설정해서 접근가능하게 한다.

page_2 = BeautifulSoup(page_1.content, 'html.parser')
page_3 = page_2.find('div', {'class' : 'xans-element- xans-product xans-product-normalpaging'}) 
# 페이지 수를 가지고 오기 위해 클래스 획득
page_4 = page_3.find_all('li', {'class' : 'xans-record-'}) # 페이지 수를 가지고 오기 위해 클래스 획득
page_5 = len(page_4) + 1

list = []
src = []

for page in range(1,page_5) : # 각 페이지를 반복해서 데이터를 수집하기 위한 반복문
    res = requests.get(page_url.format(str(page)), 
                       headers = {'user-agent' : ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                       'Chrome/71.0.3578.98 Safari/537.36')}) # 유저에이전트로 접근
    html = BeautifulSoup(res.content, 'html.parser') # html 소스 획득
    table = html.find_all('li', {'class' : 'item xans-record-'}) # 각각의 상품에 접근하기 위해 클래스획득
     
    products = []
    for i in range(len(table)) :
        prod = table[i].find('a')['href'] # 카테고리의 href를 가지고 온것 처럼 한 카테고리의 한 상품에 대한 href를 가지고 오는 것
        products.append(prod)
    
    url_list = []
    for i in range(len(products)) :
        url = base_url + products[i] # 이를 base_url과 합쳐서 각각의 상품에 접근할 수 있는 url을 획득한다.
        url_list.append(url)
    
    src = []
    for product_url in url_list :
        res = requests.get(product_url, headers = {'user-agent' : 
                                                   ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                                                       'Chrome/71.0.3578.98 Safari/537.36')}) # 주소획득
        html = BeautifulSoup(res.content, 'html.parser') # html 소스 획득
        table = html.find('div', { 'class': 'detail_slide' }) # 각 상품의 이미지를 추출하기 위한 클래스획득
    
        img = table.find('img')['src'] # 이미지의 주소에 대한 정보는 img 태그 내의 src 속성에 있다. 이를 가지고 오는 것.

# 던티비는 한 상품에 하나의 이미지만을 가지고 있다. 옷이미지와 사이즈표가 합쳐진 이미지.
        src.append(img)
                        
    for i in range(len(src)) :
        src_list = base_url + src[i] # 이미지 주소를 저장
        if src_list not in list :
            list.append(src_list)

In [20]:
list # 데이터의 개수가 많지 않다. 던티비자체 상품 개수가 적음.

['http://dontb.co.kr/web/upload/NNEditor/20181208/a.jpg',
 'http://dontb.co.kr/web/upload/NNEditor/20181204/copy-1543929464-a.jpg',
 'http://dontb.co.kr/web/upload/NNEditor/20181128/a.jpg',
 'http://dontb.co.kr/web/upload/NNEditor/20181122/a2.jpg',
 'http://dontb.co.kr/web/upload/NNEditor/20181114/a.jpg',
 'http://dontb.co.kr/web/upload/NNEditor/20181118/a.jpg',
 'http://dontb.co.kr/web/upload/NNEditor/20181111/a1.jpg',
 'http://dontb.co.kr/web/upload/NNEditor/20181106/copy-1541501973-a.jpg',
 'http://dontb.co.kr/web/upload/NNEditor/20181111/copy-1541941783-a.jpg',
 'http://dontb.co.kr/web/upload/NNEditor/20181026/a2.jpg',
 'http://dontb.co.kr/web/upload/NNEditor/20181024/a1.jpg',
 'http://dontb.co.kr/web/upload/NNEditor/20181016/copy-1539677913-a.jpg',
 'http://dontb.co.kr/web/upload/NNEditor/20181016/a1.jpg',
 'http://dontb.co.kr/web/upload/NNEditor/20181016/copy-1539694869-a.jpg',
 'http://dontb.co.kr/web/upload/NNEditor/20180910/copy-1536581849-a.jpg',
 'http://dontb.co.kr/web/uplo

In [21]:
len(list)

35

In [22]:
import urllib.request

In [24]:
for i in range(len(list)) : # 던티비의 이미지를 저장
    url = list[i]
    name = '던티비' + str(i) + '.jpg' # ex) 던티비100.jpg 형식으로 저장
    urllib.request.urlretrieve(url, name)